# ラッソ回帰の実装 (NumPy)

このノートブックでは、NumPyのみを使用してラッソ回帰をスクラッチから実装します。
ラッソ回帰は、通常の最小二乗法にL1正則化項を加えることで、一部の回帰係数を正確に0にし、変数選択を行うことができる手法です。

## 1. ライブラリのインポート

In [1]:
import numpy as np

## 2. ラッソ回帰の理論的背景

通常の線形回帰では、残差平方和 (RSS) を最小化します。
$$ L_{OLS}(\beta) = ||\mathbf{y} - \mathbf{X}\beta||^2_2 $$
ラッソ回帰は、このRSSにL1正則化項（係数の絶対値の和、L1ノルム）を加えたものを最小化します。
$$ L_{Lasso}(\beta_0, \mathbf{\beta}_{rest}) = \frac{1}{2N} ||\mathbf{y} - (\mathbf{X}_{rest}\mathbf{\beta}_{rest} + \beta_0)||^2_2 + \alpha ||\mathbf{\beta}_{rest}||_1 $$
または、単純化して以下のように書かれることもあります（$\alpha$ のスケールが変わるだけです）。
$$ L_{Lasso}(\beta_0, \mathbf{\beta}_{rest}) = \sum_{i=1}^{N} (y_i - (\beta_0 + \sum_{j=1}^{p} x_{ij}\beta_j))^2 + \alpha \sum_{j=1}^{p} |\beta_j| $$
ここでは、実装の都合上、以下の目的関数を考えます（係数 $1/2$ をRSS項の前に置くのが一般的です）。
$$ L_{Lasso}(\beta_0, \mathbf{\beta}_{rest}) = \frac{1}{2} \sum_{i=1}^{N} (y_i - (\beta_0 + \sum_{j=1}^{p} x_{ij}\beta_j))^2 + \alpha \sum_{j=1}^{p} |\beta_j| $$

ここで、
*   $N$ はサンプル数です。
*   $\beta_0$ は切片項です。
*   $\mathbf{\beta}_{rest} = (\beta_1, ..., \beta_p)^T$ は切片以外の回帰係数のベクトルです。
*   $\mathbf{X}_{rest}$ は切片項に対応する列を除いた特徴量行列です。
*   $\alpha \ge 0$ は正則化パラメータで、ペナルティの強さを調整します。
*   $||\mathbf{\beta}_{rest}||_1 = \sum_{j=1}^{p} |\beta_j|$ はL1ノルムです。

切片 $\beta_0$ は通常、正則化の対象外とします。

### 最適化: 座標降下法 (Coordinate Descent)
L1ノルムの項は微分可能でない点（$\beta_j=0$）があるため、リッジ回帰のような正規方程式で閉じた形の解を求めることができません。
代わりに、座標降下法などの反復的な最適化アルゴリズムが用いられます。

座標降下法では、一度に一つの係数 $\beta_k$ を更新し、他の係数は固定します。この処理を全ての係数に対して繰り返し行い、係数ベクトルが収束するまで続けます。

$\beta_k$ ($k \in \{1, ..., p\}$) を更新する際、他の係数を固定した目的関数を最小化します。
$\beta_k$ に関する偏導関数を0とおくと（実際には劣勾配を考える）、更新式はソフトしきい値関数 (Soft Thresholding operator) $S$ を用いて以下のように表されます。

$$ \rho_k = \sum_{i=1}^{N} x_{ik} \left( y_i - \beta_0 - \sum_{j \neq k} x_{ij}\beta_j \right) $$
$$ \beta_k \leftarrow \frac{S(\rho_k, \alpha)}{\sum_{i=1}^{N} x_{ik}^2} $$

ここで、ソフトしきい値関数 $S(z, \lambda)$ は次のように定義されます。
$$ S(z, \lambda) = \begin{cases} z - \lambda & \text{if } z > 0 \text{ and } \lambda < |z| \\ z + \lambda & \text{if } z < 0 \text{ and } \lambda < |z| \\ 0 & \text{if } \lambda \ge |z| \end{cases} $$
これは $\text{sign}(z) \max(0, |z|-\lambda)$ とも書けます。
$\sum_{i=1}^{N} x_{ik}^2$ の項は、特徴量が標準化されている場合は $N$ (または $1$、標準化の仕方による) になります。今回は標準化を外部で行うことを想定し、この項を計算に含めます。

切片 $\beta_0$ は、各イテレーションの最後に次のように更新します。
$$ \beta_0 \leftarrow \frac{1}{N} \sum_{i=1}^{N} (y_i - \sum_{j=1}^{p} x_{ij}\beta_j) $$

### $\alpha$ の効果
*   $\alpha = 0$: ラッソ回帰は通常の最小二乗法の結果に近づきます（ただし、座標降下法で解く場合、完全に一致しないこともあります）。
*   $\alpha$ が大きくなるほど、より多くの係数が0になり、モデルはスパースになります（変数選択）。
*   非常に大きな $\alpha$ では、全ての係数が0になることもあります。
適切な $\alpha$ の値は、交差検証などを用いて選択する必要があります。

## 3. データセットの準備

簡単なサンプルデータを作成してテストします。

In [2]:
# サンプルデータ
X_train_sample = np.array([
    [1, 1],
    [1, 2],
    [2, 2],
    [2, 3],
    [3, 2],
    [3, 4],
    [4, 4],
    [4, 5]
])
np.random.seed(42)
y_train_sample = 2 * X_train_sample[:, 0] + 0.5 * X_train_sample[:, 1] + np.random.normal(0, 0.5, X_train_sample.shape[0])
# 係数の一方を小さくして、スパース性が見えやすくする

print("サンプル特徴量 X_train_sample (shape):", X_train_sample.shape)
print(X_train_sample)
print("\nサンプル目的変数 y_train_sample (shape):", y_train_sample.shape)
print(y_train_sample)

サンプル特徴量 X_train_sample (shape): (8, 2)
[[1 1]
 [1 2]
 [2 2]
 [2 3]
 [3 2]
 [3 4]
 [4 4]
 [4 5]]

サンプル目的変数 y_train_sample (shape): (8,)
[ 2.74835708  2.93086785  5.32384427  6.26151493  6.88292331  7.88293152
 10.78960641 10.88371736]


### 4.1 `LassoRegression` クラスの実装

In [3]:
class LassoRegression:
    def __init__(self, alpha=1.0, n_iterations=1000, tol=1e-4):
        self.alpha = alpha              # 正則化パラメータ
        self.n_iterations = n_iterations # 座標降下法の最大反復回数
        self.tol = tol                  # 収束判定のための許容誤差
        self.intercept_ = None          # 切片 (β₀)
        self.coef_ = None               # 回帰係数 (β₁, ..., βₚ)

    def _soft_thresholding(self, rho, lambda_val):
        if rho < -lambda_val:
            return rho + lambda_val
        elif rho > lambda_val:
            return rho - lambda_val
        else:
            return 0

    def fit(self, X, y):
        '''
        訓練データを用いてモデルを学習する
        Parameters:
            X(ndarray): 特徴量行列 (サンプル数, 特徴量数). 事前に標準化されていることが望ましい。
            y(ndarray): 目的変数ベクトル (サンプル数,)
        '''
        n_samples, n_features = X.shape

        # 係数の初期化
        self.coef_ = np.zeros(n_features)
        self.intercept_ = np.mean(y) # 初期値をyの平均とする

        # 座標降下法
        for iteration in range(self.n_iterations):
            coef_old = np.copy(self.coef_)

            # 切片の更新 (イテレーションの最初または最後に更新)
            self.intercept_ = np.mean(y - X @ self.coef_)

            # 各特徴量の係数を更新
            for j in range(n_features):
                # ρ_j の計算
                # y_pred_without_j = self.intercept_ + X[:, np.arange(n_features) != j] @ self.coef_[np.arange(n_features) != j]
                # rho_j = X[:, j] @ (y - y_pred_without_j)
                
                # より直接的な計算
                current_prediction = self.intercept_ + X @ self.coef_
                residual_for_j = y - (current_prediction - X[:, j] * self.coef_[j]) # β_j * x_j の効果を除いた残差
                rho_j = X[:, j] @ residual_for_j


                # 分母の計算 (事前に計算しておくことも可能)
                sum_sq_xj = np.sum(X[:, j]**2)
                if sum_sq_xj == 0: # 特徴量の列が全て0の場合 (通常は標準化で回避)
                    self.coef_[j] = 0
                else:
                    # ソフトしきい値関数を適用
                    # 注意: ここでの alpha は目的関数の lambda に対応。
                    # 目的関数を 1/2 RSS + alpha * ||beta||_1 とした場合、ソフトしきい値の第2引数は alpha
                    self.coef_[j] = self._soft_thresholding(rho_j, self.alpha) / sum_sq_xj
            
            # 収束判定
            if np.sum(np.abs(self.coef_ - coef_old)) < self.tol:
                # print(f"Converged at iteration {iteration + 1}")
                break
        # if iteration == self.n_iterations -1:
            # print("Warning: Lasso did not converge within max_iterations.")


    def predict(self, X):
        '''
        学習済みモデルを用いて予測を行う
        Parameters:
            X(ndarray): 特徴量行列 (サンプル数, 特徴量数)
        Returns:
            ndarray: 予測値ベクトル (サンプル数,)
        '''
        if self.intercept_ is None or self.coef_ is None:
            raise ValueError("Model is not fitted yet. Call 'fit' before 'predict'.")
        
        return X @ self.coef_ + self.intercept_


## 5. モデルの学習と予測 (サンプルデータ)

異なる `alpha` の値でラッソ回帰モデルを学習させ、係数のスパース性を確認します。
ラッソ回帰は特徴量のスケールに敏感なため、通常は学習前に標準化を行いますが、
この小さなサンプルデータでは標準化せずに実行してみます。
ただし、実践では標準化が重要です。

In [4]:
# サンプルデータでテスト (標準化なし)
# 注意: 実際にはXを標準化することが推奨されます
alphas_to_test_sample = [0.1, 0.5, 1.0, 2.0, 5.0, 10.0, 20.0] # alpha=0は座標降下法では扱いにくい場合がある

print("サンプルデータでのラッソ回帰モデルの学習結果 (Xは非標準化):\n")
for alpha_val in alphas_to_test_sample:
    lasso_model_sample = LassoRegression(alpha=alpha_val, n_iterations=2000, tol=1e-5)
    lasso_model_sample.fit(X_train_sample, y_train_sample)
    
    print(f"Alpha = {alpha_val}")
    print(f"  切片 (β₀): {lasso_model_sample.intercept_:.4f}")
    print(f"  回帰係数 (β₁, β₂): {np.round(lasso_model_sample.coef_, 4)}")
    print("---")

# alpha=1.0のモデルで予測
lasso_model_sample_pred = LassoRegression(alpha=1.0, n_iterations=2000, tol=1e-5)
lasso_model_sample_pred.fit(X_train_sample, y_train_sample)
y_pred_sample = lasso_model_sample_pred.predict(X_train_sample)

print("\n訓練データに対する予測結果 (Alpha=1.0):\n")
for i in range(len(y_train_sample)):
    print(f"  実測値: {y_train_sample[i]:.2f}, 予測値: {y_pred_sample[i]:.2f}, 誤差: {y_train_sample[i] - y_pred_sample[i]:.2f}")

サンプルデータでのラッソ回帰モデルの学習結果 (Xは非標準化):

Alpha = 0.1
  切片 (β₀): 0.0554
  回帰係数 (β₁, β₂): [2.0026 0.5743]
---
Alpha = 0.5
  切片 (β₀): 0.1580
  回帰係数 (β₁, β₂): [1.9675 0.5691]
---
Alpha = 1.0
  切片 (β₀): 0.2862
  回帰係数 (β₁, β₂): [1.9237 0.5626]
---
Alpha = 2.0
  切片 (β₀): 0.5427
  回帰係数 (β₁, β₂): [1.836  0.5496]
---
Alpha = 5.0
  切片 (β₀): 1.3122
  回帰係数 (β₁, β₂): [1.5731 0.5107]
---
Alpha = 10.0
  切片 (β₀): 2.5947
  回帰係数 (β₁, β₂): [1.1347 0.4457]
---
Alpha = 20.0
  切片 (β₀): 5.1596
  回帰係数 (β₁, β₂): [0.2581 0.3159]
---

訓練データに対する予測結果 (Alpha=1.0):

  実測値: 2.75, 予測値: 2.77, 誤差: -0.02
  実測値: 2.93, 予測値: 3.34, 誤差: -0.40
  実測値: 5.32, 予測値: 5.26, 誤差: 0.07
  実測値: 6.26, 予測値: 5.82, 誤差: 0.44
  実測値: 6.88, 予測値: 7.18, 誤差: -0.30
  実測値: 7.88, 予測値: 8.31, 誤差: -0.42
  実測値: 10.79, 予測値: 10.23, 誤差: 0.56
  実測値: 10.88, 予測値: 10.79, 誤差: 0.09


## 6. より実践的なデータセットでの利用と評価 (Diabetes Dataset)

scikit-learnのDiabetesデータセットを使用して、モデルの性能を評価します。
ラッソ回帰では特徴量の標準化が特に重要です。

In [5]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

diabetes = load_diabetes()
X_diabetes, y_diabetes = diabetes.data, diabetes.target

# 特徴量の標準化
scaler_diabetes = StandardScaler()
X_diabetes_scaled = scaler_diabetes.fit_transform(X_diabetes)

# 訓練データとテストデータに分割
X_diabetes_train, X_diabetes_test, y_diabetes_train, y_diabetes_test = train_test_split(
    X_diabetes_scaled, y_diabetes, test_size=0.2, random_state=42
)

In [10]:
# Diabetesデータセットで評価
# alphaの値はデータセットによって調整が必要
# scikit-learnのLassoのalphaは 1/(2*n_samples) * RSS + alpha * ||beta||_1 のalphaに対応することが多い。
# 今回の実装のalphaは 1/2 * RSS + alpha * ||beta||_1 のalpha。
# そのため、scikit-learnと同じ効果を得るにはalphaを調整する必要がある。
# (例: scikit-learnのalpha=0.1 なら、今回のalphaは 0.1 * n_samples 程度になることがある)
# ここでは、いくつかの値を試す。
alphas_practical_lasso = [0.1, 10.0, 100, 1000] 

print("\n実践的なデータセット (Diabetes, Xは標準化済み) での評価:\n")
for alpha_val in alphas_practical_lasso:
    model_practical_lasso = LassoRegression(alpha=alpha_val, n_iterations=3000, tol=1e-5) # イテレーション数を増やす
    model_practical_lasso.fit(X_diabetes_train, y_diabetes_train)

    y_pred_diabetes_test = model_practical_lasso.predict(X_diabetes_test)

    mse_diabetes = mean_squared_error(y_diabetes_test, y_pred_diabetes_test)
    r2_diabetes = r2_score(y_diabetes_test, y_pred_diabetes_test)

    print(f"Alpha = {alpha_val}")
    print(f"  切片: {model_practical_lasso.intercept_:.4f}")
    print(f"  係数の非ゼロ要素数: {np.sum(np.abs(model_practical_lasso.coef_) > 1e-6)} / {len(model_practical_lasso.coef_)}")
    # print(f"  係数: {np.round(model_practical_lasso.coef_, 4)}") # 全係数を表示すると長いのでコメントアウト
    print(f"  平均二乗誤差 (MSE): {mse_diabetes:.4f}")
    print(f"  決定係数 (R²): {r2_diabetes:.4f}")
    print("---")


実践的なデータセット (Diabetes, Xは標準化済み) での評価:

Alpha = 0.1
  切片: 151.3456
  係数の非ゼロ要素数: 10 / 10
  平均二乗誤差 (MSE): 2900.1410
  決定係数 (R²): 0.4526
---
Alpha = 10.0
  切片: 151.3432
  係数の非ゼロ要素数: 10 / 10
  平均二乗誤差 (MSE): 2895.1848
  決定係数 (R²): 0.4535
---
Alpha = 100
  切片: 151.3429
  係数の非ゼロ要素数: 10 / 10
  平均二乗誤差 (MSE): 2873.8842
  決定係数 (R²): 0.4576
---
Alpha = 1000
  切片: 151.6634
  係数の非ゼロ要素数: 7 / 10
  平均二乗誤差 (MSE): 2800.3160
  決定係数 (R²): 0.4715
---


## 7. 考察

*   このスクラッチ実装では、ラッソ回帰の係数推定に座標降下法とソフトしきい値関数を用いました。L1正則化項により、一部の係数が正確に0になるスパースな解が得られることが特徴です。
*   **長所**:
    *   **変数選択**: 不要な特徴量の係数を0にすることで、解釈しやすく、予測に重要な特徴量を特定するのに役立ちます。
    *   **過学習の抑制**: リッジ回帰と同様に、係数の大きさを制約することで過学習を抑える効果があります。
    *   多重共線性がある場合でも、ラッソはいくつかの変数を選択し、他を0にする傾向があります（リッジは関連する変数の係数を全体的に小さくします）。

*   **短所・注意点**:
    *   **最適化アルゴリズム**: 閉じた解がないため、座標降下法のような反復解法が必要です。収束性や計算速度はリッジ回帰の正規方程式より劣る場合があります。`n_iterations` や `tol` の設定が重要です。
    *   **ハイパーパラメータ `alpha` の選択**: ラッソ回帰の性能は `alpha` の値に大きく依存します。交差検証などによる適切な選択が不可欠です。
    *   **特徴量のスケーリング**: L1ペナルティは係数の絶対値にかかるため、特徴量のスケールが異なると、スケールの大きな特徴量が不当に選択されにくくなる（またはその逆）可能性があります。学習前に特徴量を標準化することが強く推奨されます。
    *   **相関の高い変数群の扱い**: 相関の強い変数が複数ある場合、ラッソはその中から一つ（あるいは少数）の変数を選択し、他を0にする傾向があります。どの変数が選択されるかはデータやアルゴリズムの挙動に依存することがあり、安定しない場合があります。リッジ回帰はこのような場合に、相関する変数群に係数を分散させる傾向があります。
    *   **飽和**: 特徴量数 $p$ がサンプル数 $N$ より大きい場合 ($p > N$)、ラッソが選択できる非ゼロ係数の最大数は $N$ 個です。

*   **scikit-learnとの比較**:
    *   scikit-learnの `sklearn.linear_model.Lasso` は、より高度で効率的な座標降下法のソルバーを実装しており、大規模データにも対応できます。
    *   `alpha` の定義が異なる場合があります。scikit-learnでは目的関数が $\frac{1}{2N} ||y - X\beta||_2^2 + \alpha ||\beta||_1$ のように正規化されていることが多く、本実装の $\alpha$ と直接比較する際には注意が必要です。
    *   `LassoCV` のように交差検証による `alpha` の自動選択機能も備えています。本スクラッチ実装は、基本的なアルゴリズムを理解するためのものです。